In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import  numpy as np

In [ ]:
features = pd.read_csv(r'../features/features.csv')
ground_truth = pd.read_csv(r"../data/example_ground_truth.csv")


In [ ]:
ground_truth['image_id']

In [ ]:
mask = ground_truth['melanoma'] == 1

In [ ]:
melanoma_mask = ground_truth[mask]
len(melanoma_mask)

In [ ]:
total_data = []
melanoma_data = []
x = 0

for image_id in ground_truth['image_id']:
    print(image_id,x)
    image = plt.imread('../data/example_image/'+image_id+'.jpg')
    image_seg = plt.imread('../data/example_segmentation/'+image_id+'_segmentation.png')
    if image_id in list(melanoma_mask['image_id']):
        melanoma_data.append((image_id,image,image_seg))
    total_data.append((image_id,image,image_seg))
    x += 1

In [ ]:
len(melanoma_data)

In [ ]:
plt.scatter(features['area'],features['perimeter'])
plt.xlabel('area')
plt.ylabel('perimeter');

In [ ]:
melanoma_data[0][2]

In [ ]:
plt.imshow(melanoma_data[0][2])

In [ ]:
melanoma_data[0][2]

In [ ]:
def calculate_area(image):
    return np.sum(image)
calculate_area(melanoma_data[0][2])

In [ ]:
from skimage import morphology

#Structural element, that we will use as a "brush" on our mask
struct_el = morphology.disk(4)

mask_eroded = morphology.binary_erosion(melanoma_data[0][2], struct_el)

# Show side by side

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
axes[0].imshow(melanoma_data[0][2][500:1200,1000:2000], cmap='gray')
#axes[1].imshow(mask_eroded, cmap='gray')
fig.tight_layout()

# Verify it's smaller
#print(area)
print(calculate_area(melanoma_data[0][2])-np.sum(mask_eroded))
border = melanoma_data[0][2]-mask_eroded
plt.imshow(border[500:1200,1000:2000])


In [ ]:
def calculate_border(image,thickness):
    struct_el = morphology.disk(thickness)
    mask_eroded = morphology.binary_erosion(image, struct_el)
    return calculate_area(image)-np.sum(mask_eroded)
calculate_border(melanoma_data[0][2],4)

In [ ]:
plt.imshow(melanoma_data[0][1])

In [ ]:
def colour(image,mask):
    mask2 = mask == 0
    copy = image.copy()
    copy[mask2] = [0,0,0]
    #plt.imshow(copy)
    
    #print(copy[:,:,0])
    #print(copy[0])
    
    #print(len(copy[0]))
    #calculate_area(copy[0])
colour(melanoma_data[0][1],melanoma_data[0][2])